In [183]:
%load_ext autoreload
%autoreload 2

from utils import *
import numpy as np
import pandas as pd
import sys
sys.path.append("..")

from torch.utils import data

import glob
import random
import os
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns


from bokeh.io import push_notebook, show, output_notebook,reset_output
from bokeh.plotting import figure
import networkx as nx
from bokeh.plotting import figure, from_networkx, curdoc,row,column
from bokeh.models import Rect, HoverTool,  TextInput, GraphRenderer, Circle
from bokeh.io import output_file
from bokeh.plotting import figure, output_file, save
from bokeh.models import (BoxZoomTool, Circle, Line, HoverTool,
                          MultiLine, Plot, Range1d, ResetTool)
from bokeh.models import LinearColorMapper, BasicTicker, ColorBar,CustomJSHover,CustomJS
from bokeh.models import Ellipse, GraphRenderer, StaticLayoutProvider
from bokeh.models import Select, DataRange1d, ColumnDataSource, MultiChoice


import grandalf

output_notebook()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
QDIR = os.path.join("/Users/pari/LatencyCollector/queries")
JOB_QNAMES = get_qnames(os.path.join(QDIR, "job"))
CEB_QNAMES = get_qnames(os.path.join(QDIR, "ceb-small"))

MULTIPLE=False
EXP_NAME = "CEB-t2medium-gp2"
#EXP_NAME = "CEB-t2small-gp3"
#EXP_NAME = "CEB-t2medium-gp3"
#EXP_NAME = "m5ad-gp2"
#EXP_NAME = "CEB-c7large-gp2"

# ## w/ 16 processes
#EXP_NAME = "t7xlarge-gp3-c"

## w/ 9 processes
#EXP_NAME = "t7xlarge-gp3-e-stressng"
#EXP_NAMES = ["t7xlarge-gp3-c", "t7xlarge-gp3-d", "t7xlarge-gp3-e-stressng"]

#EXP_NAME = "t7xlarge-gp3-d"

if MULTIPLE:
    RESULTDIR = os.path.join("/Users/pari/prism-testbed/", "LatencyCollectorResults", "multiple", EXP_NAME)
else:
    RESULTDIR = os.path.join("/Users/pari/prism-testbed/", "LatencyCollectorResults", EXP_NAME)

#RESULTDIR = os.path.join("/Users/pari/prism-testbed/", "LatencyCollectorResults")
#print(RESULTDIR)
#CEB-t2medium-gp2

In [7]:
# print(os.path.exists(RESULTDIR))

dfs = []
# for exp in EXP_NAMES:
#     resultdir = os.path.join("/Users/pari/prism-testbed/", "LatencyCollectorResults", "multiple", exp)

print(RESULTDIR)
assert os.path.exists(RESULTDIR)

rtfns = glob.iglob(RESULTDIR + "/*/results/Runtime*.csv")
for rtfn in rtfns:
    print(rtfn)
    dfs.append(pd.read_csv(rtfn))
    
df = pd.concat(dfs)

/Users/pari/prism-testbed/LatencyCollectorResults/CEB-t2medium-gp2
/Users/pari/prism-testbed/LatencyCollectorResults/CEB-t2medium-gp2/i-0110d5685ad7c0dfe/results/Runtimes.csv


In [8]:
print(df["runtime"].sum())
print(df["runtime"].describe())

99854.79351329803
count    595.000000
mean     167.823182
std      289.108832
min        0.005181
25%        3.091356
50%       17.632031
75%      171.315723
max      909.000000
Name: runtime, dtype: float64


In [9]:
qworkloads = {}
for qname in JOB_QNAMES:
    qworkloads[qname] = "JOB"
for qname in CEB_QNAMES:
    qworkloads[qname] = "CEB"
    
df["workload"] = df.apply(lambda x: qworkloads[x["qname"]] , axis=1)
df.groupby(["workload"])["runtime"].describe(percentiles=[0.75, 0.9,0.99])

,count,mean,std,min,50%,75%,90%,99%,max
workload,,,,,,,,,
CEB,256.0,361.879533,354.644320,0.135070,249.524022,681.393749,909.000000,909.000000,909.000000
JOB,339.0,21.279154,44.066402,0.005181,5.424564,20.326218,51.011984,241.384948,415.153635


In [11]:
df["runtime"].values[0]

46.01334834098816

In [12]:
# total_times = extract_values(plan[0][0], 'Actual Total Time')
# total_times

In [13]:
import copy
def draw_plan(G):
    tmp = nx.Graph()
    for edge in G.edges():
        tmp.add_edge(edge[0], edge[1])
    pos = nx.nx_pydot.graphviz_layout(tmp, prog="dot")
    #pos = nx.nx_agraph.pygraphviz_layout(G, prog="dot")
    #print(pos)
    G = G.reverse()
    nx.draw(G, pos=pos)

In [67]:
# def get_bokeh_graph(p1, noderadius=8, title="", showplot=True,
#         height=700, width=900,
#         ):

#     G = p1
#     tmp = nx.Graph()
#     for edge in G.edges():
#         tmp.add_edge(edge[0], edge[1])
#     pos = nx.nx_pydot.graphviz_layout(tmp, prog="dot")

#     node_indices = list(p1.nodes())
#     nodevals = []

#     for nodekey in NODE_KEYS:
#         curvals = []
#         for node in node_indices:
#             if not nodekey in p1.nodes()[node]:
#                 curvals.append(-1)
#             else:
#                 curvals.append(p1.nodes()[node][nodekey])
#         nodevals.append(curvals)

#     ## adding nodes
#     graph = GraphRenderer()
#     graph.node_renderer.data_source.add(node_indices, 'index')

#     ## setting colors
#     cols = []
#     linecols = []
#     for node in node_indices:
#         linecols.append("b")
#         cols.append("blue")

#     graph.node_renderer.data_source.add(cols, 'Color')
#     graph.node_renderer.data_source.add(linecols, 'LineColor')

#     tool_list = []

#     for ci, cv in enumerate(nodevals):
#         graph.node_renderer.data_source.add(cv, NODE_KEYS[ci].replace("@", ""))
#         if "@" not in NODE_KEYS[ci]:
#             curval = "@" + NODE_KEYS[ci]
#         else:
#             curval = NODE_KEYS[ci]
#         tool_list.append(tuple([NODE_KEYS[ci], curval]))

#     graph.node_renderer.glyph = Circle(radius=noderadius,
#                                       fill_color="Color",
#                                       line_width=2,
#                                       line_color="black"
#                                       )

#     start = []
#     end = []
#     for e in p1.edges():
#         start.append(e[0])
#         end.append(e[1])

#     graph.edge_renderer.data_source.data = dict(
#                start=start,
#                end=end)

#     edgevals = []
#     for e in p1.edges():
#         if "ActualCurTime" in p1.nodes[e[1]]:
#             edgevals.append(p1.nodes[e[1]]["ActualCurTime"])
#         else:
#             edgevals.append(0.0)

#     color_mapper = LinearColorMapper(palette=get_rt_palette(),
#                                       low = min(edgevals),
#                                       high= max(edgevals))

#     graph.edge_renderer.data_source.data["line_width"] = [3 for e in p1.edges()]

#     graph.edge_renderer.data_source.data["ActualCurTime"] = edgevals
#     graph.edge_renderer.glyph.line_width = {'field': 'line_width'}
#     graph.edge_renderer.glyph.line_color = {'field': "ActualCurTime", 'transform': color_mapper}

#     graph_layout = pos
#     graph.layout_provider = StaticLayoutProvider(graph_layout=graph_layout)

#     color_bar = ColorBar(color_mapper=color_mapper,
#                      ticker= BasicTicker(),
#                      location=(0,0), title="Latency",
#                      name="latency")

#     #graph.visible = False
#     #color_bar.visible = True

#     return graph,color_bar,tool_list


In [78]:

# tmp = df[df["qname"] == "8a.sql"]
# #print(tmp["runtime"])
# #tmp = df.sample(frac=0.1)

# reset_output()
# #output_notebook()
# output_file('gp2-all-plans.html')

# FEATKEYS = ['Rows Removed by Join Filter', 'Hash Buckets', 'Hash Batches', 'Parent Relationship'
# , 'Sort Key', 'Sort Method', 'Rows Removed by Filter', 'Merge Cond', 'Node Type', 'Group Key', 'Hash Cond',
#  'Sort Space Used', 'Sort Space Type', 'Filter', 'Index Cond', 'Plan Rows', 'Recheck Cond', 'Original Hash Buckets', 
#  'Original Hash Batches', 'Heap Fetches', 'Total Cost', 'Plan Width', 'Relation Name', 'Join Filter', 'PeakMemory Usage', 
#  'Index Name', 'Exact Heap Blocks', 'Startup Cost', 'Strategy', 'Inner Unique']

# old_vals = defaultdict(list)

# plot = figure(title="Query Plan",
#     height=500, width=600,
#     tools="pan,wheel_zoom,save,reset",
#     active_scroll='wheel_zoom',
#     )

# graphs = {}

# for ri,row in tmp.iterrows():
#     if ri % 100 == 0:
#         print(ri)
    
#     exp = row["exp_analyze"]
#     qname = row["qname"] + str(ri)
    
#     try:
#         plan = eval(str(exp))
#     except Exception as e:
#         print(e)
#         continue
        
#     G = explain_to_nx(plan[0][0][0])
#     gr,col,tools = get_bokeh_graph(G, showplot=False)
    
#     plot.renderers.append(gr)
#     plot.add_layout(col, 'below')
    
#     node_hover_tool = HoverTool(tooltips=tools)
#     plot.add_tools(node_hover_tool)
    
#     graphs[qname] = (gr, col)
#     break
    
# plot.axis.visible = False
# plot.xgrid.grid_line_color = None
# plot.ygrid.grid_line_color = None

# opts = list(graphs.keys())

# select = Select(title="Plot to show:", value=opts[0], options=opts)

# select.js_on_change("value", CustomJS(args=dict(graphs=graphs, options=opts), code="""
#         for (const [key, c1] of Object.entries(graphs)) {
#             c1[0].visible = false;
#             c1[1].visible = false;
#         }
#         graphs[this.value][0].visible = true;
#         graphs[this.value][1].visible = true;
# """))

# layout = column(select, plot)
# show(layout)

In [84]:
# def get_bokeh_graph(p1, noderadius=8, title="", showplot=True,
#         height=700, width=900,
#         ):

NODE_KEYS = ['NodeType',
             'ParentRelationship',
             #'ParallelAware',
             'JoinType',
             #'Startup Cost',
             'TotalCost',
             'PlanRows', #'Plan Width',
             'ActualStartupTime',
             'ActualTotalTime',
             'ActualRows', 'ActualLoops',
             "ActualCurTime",
             #'Inner Unique',
             #'Join Filter',
             #'Rows Removed by Join Filter'
            ]

plot = figure(title="Query Plan",
    height=500, width=600,
    tools="pan,wheel_zoom,save,reset",
    active_scroll='wheel_zoom',
    )

G = explain_to_nx(plan[0][0][0])
p1 = G

tmp = nx.Graph()
for edge in G.edges():
    tmp.add_edge(edge[0], edge[1])
pos = nx.nx_pydot.graphviz_layout(tmp, prog="dot")

ordered_nodes, nodes_coordinates = zip(*sorted(pos.items()))
nodes_xs, nodes_ys = list(zip(*nodes_coordinates))

node_indices = list(p1.nodes())
node_indices.sort()

nodes_source = ColumnDataSource(dict(x=[], y=[],
                            nodes=[]
                            ))

nodes_source.data = dict(x=nodes_xs, y=nodes_ys, 
                            nodes=node_indices,
                            )


nodevals = []
for nodekey in NODE_KEYS:
    curvals = []
    for node in node_indices:
        if not nodekey in p1.nodes()[node]:
            curvals.append(-1)
        else:
            curvals.append(p1.nodes()[node][nodekey])
    nodevals.append(curvals)

tool_list = []
for ci, cv in enumerate(nodevals):
    nodes_source.add(cv, NODE_KEYS[ci].replace("@", ""))
    if "@" not in NODE_KEYS[ci]:
        curval = "@" + NODE_KEYS[ci]
    else:
        curval = NODE_KEYS[ci]
    tool_list.append(tuple([NODE_KEYS[ci], curval]))

edata = dict(xs=[],ys=[])
for u, v, data in p1.edges(data=True):
    edata['xs'].append([pos[u][0], pos[v][0]])
    edata['ys'].append([pos[u][1], pos[v][1]])
esource = ColumnDataSource(dict(xs=[], ys=[]))
esource.data = edata


r_circles = plot.circle('x', 'y', 
                  source=nodes_source,
                  #line_color = "blue",
                  line_color = "ActualCurTime",
                  line_width = 6.0,
                  size=6.0)
plain_edges_lines = plot.multi_line('xs', 'ys', 
                                    line_width=2.0,
                          color="black",
                          source=esource)

color_mapper = LinearColorMapper(palette=get_rt_palette(),
                                  low = min(edgevals),
                                  high= max(edgevals))

color_bar = ColorBar(color_mapper=color_mapper,
                 ticker= BasicTicker(),
                 location=(0,0), title="Latency",
                 name="latency")
plot.add_layout(color_bar, 'below')

node_hover_tool = HoverTool(tooltips=tool_list,
                     renderers=[r_circles], anchor="center",
                     attachment="above")


plot.add_tools(node_hover_tool)

plot.axis.visible = False
plot.xgrid.grid_line_color = None
plot.ygrid.grid_line_color = None

show(plot)

['Aggregate818811', 'Nested Loop-59915', 'Nested Loop-47689', 'Hash Join813261', 'Nested Loop854406', 'Nested Loop-93881', 'Hash Join467039', 'Seq Scan766465', 'Hash770870', 'Seq Scan914409', 'Index Scan-57378', 'Index Scan339176', 'Hash-70185', 'Seq Scan-88386', 'Index Scan624742', 'Index Scan-57385']


In [180]:
#tmp = df[df["qname"] == "8a.sql"]

def get_all_data(df):
    tmp = df.sample(frac=0.1)
    graphs = {}

    for ri,row in tmp.iterrows():
        print(ri)
        exp = row["exp_analyze"]
        qname = row["qname"] + str(ri)

        try:
            plan = eval(str(exp))
        except Exception as e:
            print(e)
            continue

        G = explain_to_nx(plan[0][0][0])
        p1 = G

        tmp = nx.Graph()
        for edge in G.edges():
            tmp.add_edge(edge[0], edge[1])
        pos = nx.nx_pydot.graphviz_layout(tmp, prog="dot")

        ordered_nodes, nodes_coordinates = zip(*sorted(pos.items()))
        nodes_xs, nodes_ys = list(zip(*nodes_coordinates))

        node_indices = list(p1.nodes())
        node_indices.sort()

        nodes_source = ColumnDataSource(dict(x=[], y=[],
                                    nodes=[]
                                    ))

        nodes_source.data = dict(x=nodes_xs, y=nodes_ys, 
                                    nodes=node_indices,
                                    )


        nodevals = []
        for nodekey in NODE_KEYS:
            curvals = []
            for node in node_indices:
                if not nodekey in p1.nodes()[node]:
                    curvals.append(-1)
                else:
                    curvals.append(p1.nodes()[node][nodekey])
            nodevals.append(curvals)

        for ci, cv in enumerate(nodevals):
            nodes_source.add(cv, NODE_KEYS[ci].replace("@", ""))
        
        nodes_source.data["MinColorValue"] = [min(nodes_source.data["ActualCurTime"])]*len(nodes_xs)
        nodes_source.data["MaxColorValue"] = [max(nodes_source.data["ActualCurTime"])]*len(nodes_xs)
        #print(nodes_source.data["MaxColorValue"])

        edata = dict(xs=[],ys=[])
        for u, v, data in p1.edges(data=True):
            edata['xs'].append([pos[u][0], pos[v][0]])
            edata['ys'].append([pos[u][1], pos[v][1]])

        esource = ColumnDataSource(dict(xs=[], ys=[]))
        esource.data = edata

        graphs[qname] = (nodes_source, esource)
    return graphs

In [181]:
nodes_source.data["MaxColorValue"][0]

12.862429

In [182]:
reset_output()
#output_notebook()
output_file('gp2-all-plans.html')

import copy

graphs = get_all_data(df)
qlist = list(graphs.keys())
nodes_source, edges_source = copy.deepcopy(graphs[qlist[0]])

plot = figure(title="Query Plan",
    height=500, width=600,
    tools="pan,wheel_zoom,save,reset",
    active_scroll='wheel_zoom',
    )

palette = get_rt_palette()
color_mapper = LinearColorMapper(palette=palette,
                                  low = min(nodes_source.data["ActualCurTime"]),
                                  high= max(nodes_source.data["ActualCurTime"]))

r_circles = plot.circle('x', 'y', 
                  source=nodes_source,
                  #line_color = "blue",
                  color={'field': "ActualCurTime", 'transform': color_mapper},
                  line_color = "blue",
                  level = "overlay",
                  line_width = 4.0,
                  size=20.0)


plain_edges_lines = plot.multi_line('xs', 'ys', 
                          line_width=2.0,
                          color="black",
                          source=edges_source)

color_bar = ColorBar(color_mapper=color_mapper,
                 ticker= BasicTicker(),
                 location=(0,0), title="Latency",
                 name="latency")

plot.add_layout(color_bar, 'below')

node_hover_tool = HoverTool(tooltips=tool_list,
                     renderers=[r_circles], anchor="center",
                     attachment="above")

plot.add_tools(node_hover_tool)

plot.axis.visible = False
plot.xgrid.grid_line_color = None
plot.ygrid.grid_line_color = None


select = Select(title="Query", value=qlist[0], options=qlist)
select.js_on_change("value", CustomJS(args=dict(nodes_source=nodes_source, 
                                                edges_source=edges_source, 
                                                graphs=graphs,select=select,
#                                                 palette = palette,
#                                                 color_bar = color_bar,
#                                                 circles = r_circles,
                                                color_mapper = color_mapper,
                                                ), code="""                                 

    for (const [key, _] of Object.entries(nodes_source.data)) {
        nodes_source.data[key] = graphs[this.value][0].data[key];
    }
    
    for (const [key, _] of Object.entries(edges_source.data)) {
        edges_source.data[key] = graphs[this.value][1].data[key];
    }
    
    // var color_mapper = new Bokeh.LinearColorMapper({palette:palette, low:0.0, high:200.0});
    console.log(graphs[this.value][0].data["MinColorValue"][0]);
    console.log(graphs[this.value][0].data["MaxColorValue"][0]);
    console.log(this.value);
    
    color_mapper.low = graphs[this.value][0].data["MinColorValue"][0];
    color_mapper.high = graphs[this.value][0].data["MaxColorValue"][0];
    // circles.glyph.fill_color = {field: "ActualCurTime", transform: color_mapper};
    
    edges_source.change.emit();
    nodes_source.change.emit();
"""))

show(column(select, plot))

570
489
name 'nan' is not defined
358
46
18
406
203
217
305
507
349
name 'nan' is not defined
517
0
219
308
286
341
313
469
77
266
369
123
583
304
435
250
423
573
278
100
59
315
3
471
name 'nan' is not defined
575
284
401
88
347
323
55
455
137
120
159
202
396
140
64
152
533
38
426
132
307
49
306
275
582
name 'nan' is not defined


In [141]:
nodes_source.data["ActualCurTime"]

[0.22520600000000013,
 0.116875,
 0.005254,
 6e-06,
 6e-06,
 8e-06,
 6e-06,
 7.099999999999999e-05,
 8e-06,
 1.390903,
 0.05826599999999999,
 0.07091100000000006,
 1.6790429999999996,
 0.388804,
 0.067712,
 0.7377229999999999,
 0.019246]

In [184]:
tool_list

[('NodeType', '@NodeType'),
 ('ParentRelationship', '@ParentRelationship'),
 ('JoinType', '@JoinType'),
 ('TotalCost', '@TotalCost'),
 ('PlanRows', '@PlanRows'),
 ('ActualStartupTime', '@ActualStartupTime'),
 ('ActualTotalTime', '@ActualTotalTime'),
 ('ActualRows', '@ActualRows'),
 ('ActualLoops', '@ActualLoops'),
 ('ActualCurTime', '@ActualCurTime')]